# Combined Data Pipeline - Cameron H
This notebook combines data cleaning, feature engineering, and modeling from codebook.ipynb and data_cleaning.ipynb


In [1]:
# IMPORTS
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import precision_score


In [2]:
# CUSTOM METRIC FUNCTIONS

def roc_auc(actual, preds):
    """Manual ROC-AUC implementation."""
    actual = np.array(actual) == 1
    tpr = []
    fpr = []
    for thresh in [x / 100.0 for x in range(0, 101)]:
        preds_t = np.array(preds) >= thresh
        tp = sum(preds_t & actual)
        fp = sum(preds_t & ~actual)
        tn = sum(~preds_t & ~actual)
        fn = sum(~preds_t & actual)
        tpr.append(tp / (tp + fn))
        fpr.append(tn / (tn + fp))

    auc = 0
    for i in range(0, len(tpr) - 1):
        auc += ((tpr[i] + tpr[i + 1]) / 2) * (fpr[i + 1] - fpr[i])
    return auc


def accuracy(actual, preds, thresh):
    """Calculate accuracy at given threshold."""
    preds = np.array(preds) >= thresh
    actual = np.array(actual) == 1
    actual = (actual == 1)
    acc = np.count_nonzero(preds == actual)/len(actual)
    return acc

def precision(actual, preds, thresh):
    """Calculate precision at given threshold."""
    preds = (np.array(preds) >= thresh)
    actual = (np.array(actual) == 1)
    tp = np.count_nonzero(preds & actual)
    fp = np.count_nonzero(preds & ~actual)
    prec = tp/(tp+fp) if (tp+fp) > 0 else 0
    return prec

def f1_score(actual, preds, thresh):
    """Calculate F1 score at given threshold."""
    preds = (np.array(preds) >= thresh)
    actual = (np.array(actual) == 1)
    tp = np.count_nonzero(preds & actual)
    fp = np.count_nonzero(preds & ~actual)
    fn = np.count_nonzero(~preds & actual)
    rec = tp/(tp+fn) if (tp+fn) > 0 else 0
    prec = tp/(tp+fp) if (tp+fp) > 0 else 0
    f1 = (2 * rec * prec)/(rec + prec) if (rec + prec) > 0 else 0
    return f1


In [3]:
# 1. LOAD DATA
# 
# DATA STRUCTURE EXPLANATION:
# - application_train_df: MAIN dataset (1 row per customer/application) - this is our base
# - bureau_df: Credit bureau history (MULTIPLE rows per customer - will be aggregated)
# - prev_app_df: Previous loan applications (MULTIPLE rows per customer - will be aggregated)
#
# PROCESS: We'll merge bureau and previous_application data INTO application_train_df
#          by aggregating them first (sum/count per customer), then adding as new columns

application_train_df = pd.read_csv('../rory_work/application_train.csv')
bureau_df = pd.read_csv('../bureau.csv')
prev_app_df = pd.read_csv('../previous_application.csv')

print(f"Application train shape: {application_train_df.shape}")
print(f"Bureau shape: {bureau_df.shape}")
print(f"Previous application shape: {prev_app_df.shape}")
print("\nNote: Bureau and Previous Application have multiple rows per customer.")
print("They will be aggregated (summed/counted) before merging into the main dataset.")


Application train shape: (307511, 122)
Bureau shape: (1716428, 17)
Previous application shape: (1670214, 37)

Note: Bureau and Previous Application have multiple rows per customer.
They will be aggregated (summed/counted) before merging into the main dataset.


In [4]:
# 2. SUBSET DATA FOR PERFORMANCE
# Take a random subset to ensure the notebook can run efficiently
# NOTE: Sampling BEFORE aggregation is fine because:
# - We'll only merge bureau/prev_app records for customers in our sample (using how='left')
# - Aggregation groups by SK_ID_CURR, so each customer gets correct aggregated values
# - Customers without bureau/prev_app records will get 0 (filled later)
SUBSET_SIZE = 30000
application_train_df = application_train_df.sample(n=min(SUBSET_SIZE, len(application_train_df)), random_state=42)

# Get list of customer IDs in our sample (for efficiency when merging)
sample_customer_ids = set(application_train_df['SK_ID_CURR'].values)

print(f"Subset application train shape: {application_train_df.shape}")
print(f"Number of unique customers in sample: {len(sample_customer_ids)}")


Subset application train shape: (30000, 122)
Number of unique customers in sample: 30000


In [5]:
# 3. DATA CLEANING

# DAYS_EMPLOYED = 365243 means "no employment record"
application_train_df["DAYS_EMPLOYED"] = application_train_df["DAYS_EMPLOYED"].replace(365243, np.nan)

# Create missingness indicators (best-practice for this dataset)
missing_cols = {}
for col in application_train_df.columns:
    missing_cols[col + "_MISSING"] = application_train_df[col].isna().astype(int)

# Add missing indicators efficiently
missing_df = pd.DataFrame(missing_cols, index=application_train_df.index)
application_train_df = pd.concat([application_train_df, missing_df], axis=1)

# Identify categorical columns
categorical_cols = application_train_df.select_dtypes(include=["object"]).columns

# Coerce numeric-like columns
numeric_like_cols = application_train_df.columns.difference(categorical_cols)
application_train_df[numeric_like_cols] = application_train_df[numeric_like_cols].apply(
    lambda col: pd.to_numeric(col, errors="coerce")
)

# Set up numeric & categorical column lists
numeric_cols = application_train_df.select_dtypes(include=[np.number]).columns
numeric_cols = numeric_cols.drop("TARGET")  # do NOT impute target

# Impute numerics (median) & categoricals (mode)
num_imputer = SimpleImputer(strategy="median")
application_train_df[numeric_cols] = num_imputer.fit_transform(application_train_df[numeric_cols])

cat_imputer = SimpleImputer(strategy="most_frequent")
application_train_df[categorical_cols] = cat_imputer.fit_transform(application_train_df[categorical_cols])

# Confirm no NaNs
print("Remaining NaNs:", application_train_df.isna().sum().sum())
print(f"Shape after cleaning: {application_train_df.shape}")


Remaining NaNs: 0
Shape after cleaning: (30000, 244)


In [6]:
# 4. FEATURE ENGINEERING - BUREAU DATA
# 
# Bureau data has multiple credit records per customer. We aggregate them into summary features:
# - Total overdue amount per customer
# - Total debt per customer  
# - Total times credit was prolonged per customer
# - Total days overdue per customer
# Then we add these as NEW COLUMNS to the main dataset (one value per customer)

# Filter bureau data to only include records for customers in our sample (for efficiency)
bureau_filtered = bureau_df[bureau_df['SK_ID_CURR'].isin(sample_customer_ids)].copy()

# Merge with bureau data (creates temporary merged df with multiple rows per customer)
application_bureau_df = pd.merge(application_train_df, bureau_filtered, on="SK_ID_CURR", how="left")

# Aggregate bureau features
total_overdue = application_bureau_df.groupby("SK_ID_CURR")["AMT_CREDIT_SUM_OVERDUE"].sum()
total_debt = application_bureau_df.groupby("SK_ID_CURR")["AMT_CREDIT_SUM_DEBT"].sum()
times_prolonged = application_bureau_df.groupby("SK_ID_CURR")["CNT_CREDIT_PROLONG"].sum()
days_overdue = application_bureau_df.groupby("SK_ID_CURR")["CREDIT_DAY_OVERDUE"].sum()

# Merge aggregated features back
application_train_merged_df = application_train_df.merge(total_overdue, on='SK_ID_CURR', how='left')
application_train_merged_df = application_train_merged_df.merge(total_debt, on='SK_ID_CURR', how='left')
application_train_merged_df = application_train_merged_df.merge(times_prolonged, on='SK_ID_CURR', how='left')
application_train_merged_df = application_train_merged_df.merge(days_overdue, on="SK_ID_CURR", how='left')

# Fill NaN values with 0 for new features
application_train_merged_df['AMT_CREDIT_SUM_OVERDUE'] = application_train_merged_df['AMT_CREDIT_SUM_OVERDUE'].fillna(0)
application_train_merged_df['AMT_CREDIT_SUM_DEBT'] = application_train_merged_df['AMT_CREDIT_SUM_DEBT'].fillna(0)
application_train_merged_df['CNT_CREDIT_PROLONG'] = application_train_merged_df['CNT_CREDIT_PROLONG'].fillna(0)
application_train_merged_df['CREDIT_DAY_OVERDUE'] = application_train_merged_df['CREDIT_DAY_OVERDUE'].fillna(0)

print(f"Shape after bureau merge: {application_train_merged_df.shape}")


Shape after bureau merge: (30000, 248)


In [7]:
# 5. FEATURE ENGINEERING - PREVIOUS APPLICATION DATA
#
# Previous application data has multiple applications per customer. We create summary features:
# - PREV_APPS: Count of total previous applications per customer
# - NUM_APPROVED: Count of approved previous applications per customer
# Then we add these as NEW COLUMNS to the main dataset (one value per customer)

# Filter previous application data to only include records for customers in our sample (for efficiency)
prev_app_filtered = prev_app_df[prev_app_df['SK_ID_CURR'].isin(sample_customer_ids)].copy()

# Count previous applications per customer
prev_app_ct = prev_app_filtered[["SK_ID_CURR"]].copy()
prev_app_ct["PREV_APPS"] = 1
prev_app_ct = prev_app_ct.groupby("SK_ID_CURR")["PREV_APPS"].count().reset_index()

# Count approved previous applications
if "NAME_CONTRACT_STATUS" in prev_app_filtered.columns:
    prev_app_approved = prev_app_filtered[["SK_ID_CURR", "NAME_CONTRACT_STATUS"]].copy()
    prev_app_approved["NUM_APPROVED"] = np.where(prev_app_approved["NAME_CONTRACT_STATUS"] == "Approved", 1, 0)
    prev_app_approved = prev_app_approved.groupby("SK_ID_CURR")["NUM_APPROVED"].sum().reset_index()
    
    # Merge counts
    prev_app_ct = prev_app_ct.merge(prev_app_approved, on="SK_ID_CURR", how="left")
    prev_app_ct["NUM_APPROVED"] = prev_app_ct["NUM_APPROVED"].fillna(0)
else:
    # If NAME_CONTRACT_STATUS doesn't exist, just use PREV_APPS
    prev_app_ct["NUM_APPROVED"] = 0

# Merge with main dataframe
application_train_merged_df = application_train_merged_df.merge(prev_app_ct, on="SK_ID_CURR", how="left")
application_train_merged_df['PREV_APPS'] = application_train_merged_df['PREV_APPS'].fillna(0)
application_train_merged_df['NUM_APPROVED'] = application_train_merged_df['NUM_APPROVED'].fillna(0)

print(f"Shape after previous application merge: {application_train_merged_df.shape}")


Shape after previous application merge: (30000, 250)


In [8]:
# 6. STANDARDIZE NEW FEATURES

scaler = StandardScaler()
cols_to_standardize = ['AMT_CREDIT_SUM_OVERDUE', 'AMT_CREDIT_SUM_DEBT', 'CNT_CREDIT_PROLONG', 
                       'CREDIT_DAY_OVERDUE', 'PREV_APPS', 'NUM_APPROVED']

# Only standardize columns that exist
cols_to_standardize = [col for col in cols_to_standardize if col in application_train_merged_df.columns]

if cols_to_standardize:
    application_train_merged_df[cols_to_standardize] = scaler.fit_transform(application_train_merged_df[cols_to_standardize])
    print(f"Standardized columns: {cols_to_standardize}")
else:
    print("No columns to standardize")


Standardized columns: ['AMT_CREDIT_SUM_OVERDUE', 'AMT_CREDIT_SUM_DEBT', 'CNT_CREDIT_PROLONG', 'CREDIT_DAY_OVERDUE', 'PREV_APPS', 'NUM_APPROVED']


In [9]:
# 7. ONE-HOT ENCODE CATEGORICAL FIELDS

df_encoded = pd.get_dummies(application_train_merged_df, drop_first=True)
print(f"Shape after one-hot encoding: {df_encoded.shape}")


Shape after one-hot encoding: (30000, 357)


In [10]:
# 8. CORRELATION-BASED FILTERING

# Only compute correlations on numeric columns
numeric_df = df_encoded.select_dtypes(include=[np.number])
correlations = numeric_df.corr()["TARGET"]

# Filter out low correlation columns
low_corr_cols = correlations[abs(correlations) < 0.01].index.tolist()

# DO NOT drop TARGET even if correlation calculation returns it
low_corr_cols = [col for col in low_corr_cols if col != "TARGET"]

df_filtered = df_encoded.drop(columns=low_corr_cols, errors='ignore')

print(f"Dropped {len(low_corr_cols)} low correlation columns")
print(f"Final shape: {df_filtered.shape}")


Dropped 34 low correlation columns
Final shape: (30000, 323)


In [11]:
# 9. PREPARE FOR MODELING

# Remove any remaining NaNs
df_filtered = df_filtered.dropna()

X = df_filtered.drop("TARGET", axis=1)
y = df_filtered["TARGET"]

print(f"Final X shape: {X.shape}")
print(f"Final y shape: {y.shape}")
print(f"Target distribution: {y.value_counts().to_dict()}")
print(f"Target proportion - Class 0: {y.value_counts()[0]/len(y):.3f}, Class 1: {y.value_counts()[1]/len(y):.3f}")


Final X shape: (30000, 322)
Final y shape: (30000,)
Target distribution: {0: 27543, 1: 2457}
Target proportion - Class 0: 0.918, Class 1: 0.082


In [12]:
# 10. DATA SPLITTING FUNCTIONS (FROM SCRATCH)
# Assignment requirement: Random split, Stratified split, Non-random split

def random_split(X, y, test_size=0.2, random_state=42):
    """
    Random train/test split implementation from scratch.
    
    Parameters:
    - X: Features dataframe
    - y: Target series
    - test_size: Proportion of data for test set (default 0.2)
    - random_state: Random seed for reproducibility
    
    Returns:
    - X_train, X_test, y_train, y_test
    """
    np.random.seed(random_state)
    n_samples = len(X)
    n_test = int(n_samples * test_size)
    
    # Create random indices
    indices = np.arange(n_samples)
    np.random.shuffle(indices)
    
    # Split indices
    test_indices = indices[:n_test]
    train_indices = indices[n_test:]
    
    # Split data
    X_train = X.iloc[train_indices].reset_index(drop=True)
    X_test = X.iloc[test_indices].reset_index(drop=True)
    y_train = y.iloc[train_indices].reset_index(drop=True)
    y_test = y.iloc[test_indices].reset_index(drop=True)
    
    return X_train, X_test, y_train, y_test


def stratified_split(X, y, test_size=0.2, random_state=42):
    """
    Stratified train/test split implementation from scratch.
    Maintains class proportions in both train and test sets.
    
    Parameters:
    - X: Features dataframe
    - y: Target series
    - test_size: Proportion of data for test set (default 0.2)
    - random_state: Random seed for reproducibility
    
    Returns:
    - X_train, X_test, y_train, y_test
    """
    np.random.seed(random_state)
    
    # Get indices for each class
    class_0_indices = np.where(y == 0)[0]
    class_1_indices = np.where(y == 1)[0]
    
    # Shuffle each class's indices
    np.random.shuffle(class_0_indices)
    np.random.shuffle(class_1_indices)
    
    # Calculate test size for each class
    n_test_0 = int(len(class_0_indices) * test_size)
    n_test_1 = int(len(class_1_indices) * test_size)
    
    # Split indices for each class
    test_indices_0 = class_0_indices[:n_test_0]
    train_indices_0 = class_0_indices[n_test_0:]
    
    test_indices_1 = class_1_indices[:n_test_1]
    train_indices_1 = class_1_indices[n_test_1:]
    
    # Combine indices
    test_indices = np.concatenate([test_indices_0, test_indices_1])
    train_indices = np.concatenate([train_indices_0, train_indices_1])
    
    # Shuffle combined indices
    np.random.shuffle(test_indices)
    np.random.shuffle(train_indices)
    
    # Split data
    X_train = X.iloc[train_indices].reset_index(drop=True)
    X_test = X.iloc[test_indices].reset_index(drop=True)
    y_train = y.iloc[train_indices].reset_index(drop=True)
    y_test = y.iloc[test_indices].reset_index(drop=True)
    
    return X_train, X_test, y_train, y_test


def non_random_split(X, y, test_size=0.2):
    """
    Non-random train/test split implementation from scratch.
    Uses first (1-test_size) for training, last test_size for testing.
    This simulates temporal split or preserves data order.
    
    Parameters:
    - X: Features dataframe
    - y: Target series
    - test_size: Proportion of data for test set (default 0.2)
    
    Returns:
    - X_train, X_test, y_train, y_test
    """
    n_samples = len(X)
    n_test = int(n_samples * test_size)
    n_train = n_samples - n_test
    
    # Split by order (first n_train for train, last n_test for test)
    train_indices = np.arange(n_train)
    test_indices = np.arange(n_train, n_samples)
    
    # Split data
    X_train = X.iloc[train_indices].reset_index(drop=True)
    X_test = X.iloc[test_indices].reset_index(drop=True)
    y_train = y.iloc[train_indices].reset_index(drop=True)
    y_test = y.iloc[test_indices].reset_index(drop=True)
    
    return X_train, X_test, y_train, y_test


# Test the split functions
print("Testing split functions...")
X_train_rand, X_test_rand, y_train_rand, y_test_rand = random_split(X, y, test_size=0.2, random_state=42)
X_train_strat, X_test_strat, y_train_strat, y_test_strat = stratified_split(X, y, test_size=0.2, random_state=42)
X_train_nonrand, X_test_nonrand, y_train_nonrand, y_test_nonrand = non_random_split(X, y, test_size=0.2)

print(f"\nRandom Split:")
print(f"  Train: {len(X_train_rand)} samples, Class 0: {(y_train_rand==0).sum()}, Class 1: {(y_train_rand==1).sum()}")
print(f"  Test:  {len(X_test_rand)} samples, Class 0: {(y_test_rand==0).sum()}, Class 1: {(y_test_rand==1).sum()}")

print(f"\nStratified Split:")
print(f"  Train: {len(X_train_strat)} samples, Class 0: {(y_train_strat==0).sum()}, Class 1: {(y_train_strat==1).sum()}")
print(f"  Test:  {len(X_test_strat)} samples, Class 0: {(y_test_strat==0).sum()}, Class 1: {(y_test_strat==1).sum()}")

print(f"\nNon-Random Split:")
print(f"  Train: {len(X_train_nonrand)} samples, Class 0: {(y_train_nonrand==0).sum()}, Class 1: {(y_train_nonrand==1).sum()}")
print(f"  Test:  {len(X_test_nonrand)} samples, Class 0: {(y_test_nonrand==0).sum()}, Class 1: {(y_test_nonrand==1).sum()}")


Testing split functions...

Random Split:
  Train: 24000 samples, Class 0: 22007, Class 1: 1993
  Test:  6000 samples, Class 0: 5536, Class 1: 464

Stratified Split:
  Train: 24001 samples, Class 0: 22035, Class 1: 1966
  Test:  5999 samples, Class 0: 5508, Class 1: 491

Non-Random Split:
  Train: 24000 samples, Class 0: 22031, Class 1: 1969
  Test:  6000 samples, Class 0: 5512, Class 1: 488


In [13]:
# 12B. AUTOMATIC SPLIT STRATEGY COMPARISON
# Assignment requirement: "How does changing split strategy impact performance?"

# This cell runs all 3 models on all 3 split strategies and compares results
# This answers: "Which split approach makes you most confident about Kaggle performance?"

print("=" * 70)
print("COMPREHENSIVE SPLIT STRATEGY COMPARISON")
print("=" * 70)
print("\nThis will train all 3 models on all 3 split strategies...")
print("Total: 3 models × 3 splits = 9 model trainings\n")

# Store results for comparison
comparison_results = []

# Define split strategies
split_strategies = {
    'random': random_split,
    'stratified': stratified_split,
    'non_random': non_random_split
}

# Loop through each split strategy
for split_name, split_func in split_strategies.items():
    print(f"\n{'='*70}")
    print(f"SPLIT STRATEGY: {split_name.upper()}")
    print(f"{'='*70}")
    
    # Create split (non_random_split doesn't accept random_state)
    if split_name == 'non_random':
        X_train_split, X_test_split, y_train_split, y_test_split = split_func(
            X, y, test_size=0.2
        )
    else:
        X_train_split, X_test_split, y_train_split, y_test_split = split_func(
            X, y, test_size=0.2, random_state=42
        )
    
    print(f"Train: {len(X_train_split)} samples | Test: {len(X_test_split)} samples")
    print(f"Train class dist: 0={(y_train_split==0).sum()}, 1={(y_train_split==1).sum()}")
    print(f"Test class dist:  0={(y_test_split==0).sum()}, 1={(y_test_split==1).sum()}")
    
    # Initialize models for this split
    models = {
        "Logistic Regression": LogisticRegression(max_iter=10000),
        "SVM": svm.SVC(probability=True),
        "LDA": LinearDiscriminantAnalysis()
    }
    
    # Train and evaluate each model
    for model_name, model in models.items():
        print(f"\n  Training {model_name}...")
        
        # Train
        model.fit(X_train_split, y_train_split)
        
        # Get predictions
        y_pred_proba_test = model.predict_proba(X_test_split)[:, 1]
        y_pred_proba_train = model.predict_proba(X_train_split)[:, 1]
        
        # Calculate metrics on test set
        roc_auc_test = roc_auc(y_test_split, y_pred_proba_test)
        roc_auc_train = roc_auc(y_train_split, y_pred_proba_train)
        
        threshold = 0.5
        accuracy_test = accuracy(y_test_split, y_pred_proba_test, threshold)
        precision_test = precision(y_test_split, y_pred_proba_test, threshold)
        f1_test = f1_score(y_test_split, y_pred_proba_test, threshold)
        
        # Store results
        comparison_results.append({
            'Split Strategy': split_name,
            'Model': model_name,
            'ROC-AUC (Test)': roc_auc_test,
            'ROC-AUC (Train)': roc_auc_train,
            'Accuracy (Test)': accuracy_test,
            'Precision (Test)': precision_test,
            'F1 (Test)': f1_test,
            'Train Size': len(X_train_split),
            'Test Size': len(X_test_split)
        })
        
        print(f"    Test ROC-AUC: {roc_auc_test:.4f}")

# Create comparison dataframe
comparison_df = pd.DataFrame(comparison_results)

print(f"\n{'='*70}")
print("COMPARISON SUMMARY")
print(f"{'='*70}\n")
print(comparison_df.to_string(index=False))


COMPREHENSIVE SPLIT STRATEGY COMPARISON

This will train all 3 models on all 3 split strategies...
Total: 3 models × 3 splits = 9 model trainings


SPLIT STRATEGY: RANDOM
Train: 24000 samples | Test: 6000 samples
Train class dist: 0=22007, 1=1993
Test class dist:  0=5536, 1=464

  Training Logistic Regression...


c:\Users\Panda\OneDrive\Desktop\Data 403 - Project 2\DATA403-Project-2\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 10000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=10000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


    Test ROC-AUC: 0.6209

  Training SVM...
    Test ROC-AUC: 0.4999

  Training LDA...
    Test ROC-AUC: 0.7460

SPLIT STRATEGY: STRATIFIED
Train: 24001 samples | Test: 5999 samples
Train class dist: 0=22035, 1=1966
Test class dist:  0=5508, 1=491

  Training Logistic Regression...


c:\Users\Panda\OneDrive\Desktop\Data 403 - Project 2\DATA403-Project-2\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 10000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=10000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


    Test ROC-AUC: 0.6662

  Training SVM...
    Test ROC-AUC: 0.5002

  Training LDA...
    Test ROC-AUC: 0.7649

SPLIT STRATEGY: NON_RANDOM
Train: 24000 samples | Test: 6000 samples
Train class dist: 0=22031, 1=1969
Test class dist:  0=5512, 1=488

  Training Logistic Regression...


c:\Users\Panda\OneDrive\Desktop\Data 403 - Project 2\DATA403-Project-2\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 10000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=10000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


    Test ROC-AUC: 0.6286

  Training SVM...
    Test ROC-AUC: 0.5003

  Training LDA...
    Test ROC-AUC: 0.7355

COMPARISON SUMMARY

Split Strategy               Model  ROC-AUC (Test)  ROC-AUC (Train)  Accuracy (Test)  Precision (Test)  F1 (Test)  Train Size  Test Size
        random Logistic Regression        0.620860         0.648513         0.922667          0.000000   0.000000       24000       6000
        random                 SVM        0.499910         0.500319         0.922667          0.000000   0.000000       24000       6000
        random                 LDA        0.746042         0.767503         0.921333          0.409091   0.070866       24000       6000
    stratified Logistic Regression        0.666194         0.651370         0.918153          0.000000   0.000000       24001       5999
    stratified                 SVM        0.500182         0.500667         0.918153          0.000000   0.000000       24001       5999
    stratified                 LDA        0.

In [14]:
# 12C. SPLIT STRATEGY ANALYSIS
# Analyze which split strategy performs best for each model

print("=" * 70)
print("SPLIT STRATEGY ANALYSIS")
print("=" * 70)

# Group by model and show best split strategy
for model_name in comparison_df['Model'].unique():
    model_data = comparison_df[comparison_df['Model'] == model_name]
    best_split = model_data.loc[model_data['ROC-AUC (Test)'].idxmax()]
    
    print(f"\n{model_name}:")
    print(f"  Best split: {best_split['Split Strategy']} (ROC-AUC: {best_split['ROC-AUC (Test)']:.4f})")
    print(f"  All splits:")
    for _, row in model_data.iterrows():
        print(f"    {row['Split Strategy']:12s}: ROC-AUC = {row['ROC-AUC (Test)']:.4f} "
              f"(Train: {row['ROC-AUC (Train)']:.4f}, Diff: {row['ROC-AUC (Train)'] - row['ROC-AUC (Test)']:.4f})")

# Overall best split strategy (average across all models)
split_avg = comparison_df.groupby('Split Strategy')['ROC-AUC (Test)'].mean().sort_values(ascending=False)
print(f"\n{'='*70}")
print("OVERALL BEST SPLIT STRATEGY (Average ROC-AUC across all models):")
print(f"{'='*70}")
for split_name, avg_score in split_avg.items():
    print(f"  {split_name:12s}: {avg_score:.4f}")

print(f"\n{'='*70}")
print("RECOMMENDATION:")
print(f"{'='*70}")
best_overall = split_avg.index[0]
print(f"Use '{best_overall}' split strategy for final modeling")
print(f"(Average ROC-AUC: {split_avg.iloc[0]:.4f} across all 3 models)")


SPLIT STRATEGY ANALYSIS

Logistic Regression:
  Best split: stratified (ROC-AUC: 0.6662)
  All splits:
    random      : ROC-AUC = 0.6209 (Train: 0.6485, Diff: 0.0277)
    stratified  : ROC-AUC = 0.6662 (Train: 0.6514, Diff: -0.0148)
    non_random  : ROC-AUC = 0.6286 (Train: 0.6489, Diff: 0.0203)

SVM:
  Best split: non_random (ROC-AUC: 0.5003)
  All splits:
    random      : ROC-AUC = 0.4999 (Train: 0.5003, Diff: 0.0004)
    stratified  : ROC-AUC = 0.5002 (Train: 0.5007, Diff: 0.0005)
    non_random  : ROC-AUC = 0.5003 (Train: 0.5008, Diff: 0.0005)

LDA:
  Best split: stratified (ROC-AUC: 0.7649)
  All splits:
    random      : ROC-AUC = 0.7460 (Train: 0.7675, Diff: 0.0215)
    stratified  : ROC-AUC = 0.7649 (Train: 0.7629, Diff: -0.0020)
    non_random  : ROC-AUC = 0.7355 (Train: 0.7694, Diff: 0.0339)

OVERALL BEST SPLIT STRATEGY (Average ROC-AUC across all models):
  stratified  : 0.6437
  random      : 0.6223
  non_random  : 0.6215

RECOMMENDATION:
Use 'stratified' split strategy 

In [15]:
# 11. CROSS-VALIDATION IMPLEMENTATION (FROM SCRATCH)
# Assignment requirement: Implement cross-validation from scratch

def k_fold_cross_validation(X, y, k=5, random_state=42, stratified=True):
    """
    K-fold cross-validation implementation from scratch.
    
    Parameters:
    - X: Features dataframe
    - y: Target series
    - k: Number of folds (default 5)
    - random_state: Random seed for reproducibility
    - stratified: If True, use stratified k-fold (maintains class proportions)
    
    Returns:
    - List of tuples: [(train_indices, test_indices), ...] for k folds
    """
    np.random.seed(random_state)
    n_samples = len(X)
    fold_size = n_samples // k
    
    if stratified:
        # Stratified k-fold: maintain class proportions in each fold
        class_0_indices = np.where(y == 0)[0]
        class_1_indices = np.where(y == 1)[0]
        
        np.random.shuffle(class_0_indices)
        np.random.shuffle(class_1_indices)
        
        # Calculate fold sizes for each class
        fold_size_0 = len(class_0_indices) // k
        fold_size_1 = len(class_1_indices) // k
        
        folds = []
        for i in range(k):
            # Get indices for this fold from each class
            start_0 = i * fold_size_0
            end_0 = (i + 1) * fold_size_0 if i < k - 1 else len(class_0_indices)
            
            start_1 = i * fold_size_1
            end_1 = (i + 1) * fold_size_1 if i < k - 1 else len(class_1_indices)
            
            test_indices = np.concatenate([
                class_0_indices[start_0:end_0],
                class_1_indices[start_1:end_1]
            ])
            train_indices = np.concatenate([
                class_0_indices[:start_0],
                class_0_indices[end_0:],
                class_1_indices[:start_1],
                class_1_indices[end_1:]
            ])
            
            np.random.shuffle(test_indices)
            np.random.shuffle(train_indices)
            
            folds.append((train_indices, test_indices))
    else:
        # Regular k-fold: random split
        indices = np.arange(n_samples)
        np.random.shuffle(indices)
        
        folds = []
        for i in range(k):
            start = i * fold_size
            end = (i + 1) * fold_size if i < k - 1 else n_samples
            
            test_indices = indices[start:end]
            train_indices = np.concatenate([indices[:start], indices[end:]])
            
            folds.append((train_indices, test_indices))
    
    return folds


def evaluate_with_cv(X, y, model, metric_func, k=5, random_state=42, stratified=True):
    """
    Evaluate a model using k-fold cross-validation.
    
    Parameters:
    - X: Features dataframe
    - y: Target series
    - model: Model object with fit() and predict_proba() methods
    - metric_func: Function that takes (y_true, y_pred_proba) and returns metric value
    - k: Number of folds
    - random_state: Random seed
    - stratified: Use stratified k-fold
    
    Returns:
    - List of metric values for each fold
    """
    folds = k_fold_cross_validation(X, y, k=k, random_state=random_state, stratified=stratified)
    scores = []
    
    for fold_idx, (train_indices, test_indices) in enumerate(folds):
        # Split data for this fold
        X_train_fold = X.iloc[train_indices].reset_index(drop=True)
        X_test_fold = X.iloc[test_indices].reset_index(drop=True)
        y_train_fold = y.iloc[train_indices].reset_index(drop=True)
        y_test_fold = y.iloc[test_indices].reset_index(drop=True)
        
        # Train model
        model.fit(X_train_fold, y_train_fold)
        
        # Get predictions
        y_pred_proba = model.predict_proba(X_test_fold)[:, 1]
        
        # Calculate metric
        score = metric_func(y_test_fold, y_pred_proba)
        scores.append(score)
    
    return scores


# Test cross-validation
print("Testing cross-validation implementation...")
print(f"Created {5}-fold cross-validation splits")
print("Ready to use for model evaluation")


Testing cross-validation implementation...
Created 5-fold cross-validation splits
Ready to use for model evaluation


In [16]:
# 12. COMPARE SPLIT STRATEGIES
# Assignment requirement: "How does changing split strategy impact performance?"

# We'll train models on each split and compare validation performance
# For now, we'll use one split strategy for the main modeling
# Full comparison will be done after models are trained

# Choose which split to use for main modeling
# Options: 'random', 'stratified', 'non_random'
SPLIT_STRATEGY = 'stratified'  # Can change this to compare

if SPLIT_STRATEGY == 'random':
    X_train, X_test, y_train, y_test = random_split(X, y, test_size=0.2, random_state=42)
elif SPLIT_STRATEGY == 'stratified':
    X_train, X_test, y_train, y_test = stratified_split(X, y, test_size=0.2, random_state=42)
else:  # non_random
    X_train, X_test, y_train, y_test = non_random_split(X, y, test_size=0.2)

print(f"Using {SPLIT_STRATEGY} split strategy")
print(f"Train set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")
print(f"Train class distribution - Class 0: {(y_train==0).sum()}, Class 1: {(y_train==1).sum()}")
print(f"Test class distribution - Class 0: {(y_test==0).sum()}, Class 1: {(y_test==1).sum()}")


Using stratified split strategy
Train set: 24001 samples
Test set: 5999 samples
Train class distribution - Class 0: 22035, Class 1: 1966
Test class distribution - Class 0: 5508, Class 1: 491


In [17]:
# 13. TRAIN MODELS (ON TRAINING SET)

model1 = LogisticRegression(max_iter=10000)
model2 = svm.SVC(probability=True)
model3 = LinearDiscriminantAnalysis()

print("Training Logistic Regression...")
model1.fit(X_train, y_train)

print("Training SVM...")
model2.fit(X_train, y_train)

print("Training LDA...")
model3.fit(X_train, y_train)

print("All models trained successfully on training set!")


Training Logistic Regression...


c:\Users\Panda\OneDrive\Desktop\Data 403 - Project 2\DATA403-Project-2\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 10000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=10000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Training SVM...
Training LDA...
All models trained successfully on training set!


In [18]:
# 14. EVALUATE MODELS ON TEST SET

# Get predictions on test set
prob_predictions_test = {
    "Logistic Regression": model1.predict_proba(X_test)[:, 1],
    "SVM": model2.predict_proba(X_test)[:, 1],
    "LDA": model3.predict_proba(X_test)[:, 1],
}

# Also get predictions on training set for comparison
prob_predictions_train = {
    "Logistic Regression": model1.predict_proba(X_train)[:, 1],
    "SVM": model2.predict_proba(X_train)[:, 1],
    "LDA": model3.predict_proba(X_train)[:, 1],
}


In [19]:
# 15. ROC-AUC EVALUATION (TEST SET)

roc_auc_rows_test = []
for name, probs in prob_predictions_test.items():
    roc_auc_rows_test.append({
        "Model": name,
        "ROC-AUC (Test)": roc_auc(y_test, probs),
    })

roc_auc_df_test = pd.DataFrame(roc_auc_rows_test)
print("=== ROC-AUC on TEST SET ===")
print(roc_auc_df_test)

# Also evaluate on training set for comparison
roc_auc_rows_train = []
for name, probs in prob_predictions_train.items():
    roc_auc_rows_train.append({
        "Model": name,
        "ROC-AUC (Train)": roc_auc(y_train, probs),
    })

roc_auc_df_train = pd.DataFrame(roc_auc_rows_train)
print("\n=== ROC-AUC on TRAINING SET (for comparison) ===")
print(roc_auc_df_train)


=== ROC-AUC on TEST SET ===
                 Model  ROC-AUC (Test)
0  Logistic Regression        0.666194
1                  SVM        0.500182
2                  LDA        0.764865

=== ROC-AUC on TRAINING SET (for comparison) ===
                 Model  ROC-AUC (Train)
0  Logistic Regression         0.651370
1                  SVM         0.500667
2                  LDA         0.762854


In [20]:
# 16. ADDITIONAL METRICS (ACCURACY, PRECISION, F1) - TEST SET

threshold = 0.5

metric_rows_test = []
for name, probs in prob_predictions_test.items():
    preds_binary = (probs >= threshold).astype(int)
    metric_rows_test.append({
        "Model": name,
        "Accuracy (custom)": accuracy(y_test, probs, threshold),
        "Precision (custom)": precision(y_test, probs, threshold),
        "F1 (custom)": f1_score(y_test, probs, threshold),
        "Precision (sklearn)": precision_score(y_test, preds_binary, zero_division=0),
    })

metrics_df_test = pd.DataFrame(metric_rows_test)
print("=== METRICS on TEST SET ===")
print(metrics_df_test)


=== METRICS on TEST SET ===
                 Model  Accuracy (custom)  Precision (custom)  F1 (custom)  \
0  Logistic Regression           0.918153            0.000000     0.000000   
1                  SVM           0.918153            0.000000     0.000000   
2                  LDA           0.915986            0.341463     0.052632   

   Precision (sklearn)  
0             0.000000  
1             0.000000  
2             0.341463  


In [21]:
# 17. SUMMARY - TEST SET PERFORMANCE

# Combine all metrics for test set
summary_df_test = roc_auc_df_test.merge(metrics_df_test, on="Model")
print("\n=== FINAL MODEL COMPARISON (TEST SET) ===")
print(summary_df_test.to_string(index=False))

# Note: This evaluation is on the test set using the selected split strategy
# To compare split strategies, re-run with different SPLIT_STRATEGY values in cell 12



=== FINAL MODEL COMPARISON (TEST SET) ===
              Model  ROC-AUC (Test)  Accuracy (custom)  Precision (custom)  F1 (custom)  Precision (sklearn)
Logistic Regression        0.666194           0.918153            0.000000     0.000000             0.000000
                SVM        0.500182           0.918153            0.000000     0.000000             0.000000
                LDA        0.764865           0.915986            0.341463     0.052632             0.341463
